Inga Ulusoy, Computational modelling in python, SoSe2020 

# Quantum dynamics and the time-dependent Schrodinger equation: The particle in a box II

## Stationary states and superposition states

The energy eigenstates in Problem10 are called *stationary states* as they fulfill the following property:
\begin{align}
P(x,t) = |\Psi(x,t)|^2 dx = |\Psi(x)|^2 dx = P(x)
\end{align}
The probability distibution $P(x,t)$ is constant over time, and with that, the expectation values and properties of the system.

If we want to obtain the time-dependence of the energy eigenstates, we insert them into the time-dependent Schrodinger equation (TDSE), assuming that
\begin{align}
\Psi(x,t) =  \Psi(x) \theta(t)
\end{align}
as the time and space variables are separable for a time-independent Hamiltonian. We obtain
\begin{align}
i \hbar \frac{\partial}{\partial t} \Psi(x) \theta(t) = \hat{H} \Psi(x) \theta(t)
\end{align}
We know that
\begin{align}
\hat{H} \Psi(x) = E \Psi(x)
\end{align}
and can therefore write
\begin{align}
i \hbar \Psi(x) \frac{\partial}{\partial t} \theta(t) = E \Psi(x) \theta(t)
\end{align}
Now we divide by $\Psi(x)$ as no operator acts on it
\begin{align}
i \hbar \frac{\partial}{\partial t} \theta(t) = E \theta(t)
\end{align}
It is clear to see that $\theta(t) = \exp{-i E t/\hbar}$ is a valid solution. The complete wave function thus reads
\begin{align}
\Psi(x,t) = \Psi(x) \exp\left(-iEt/\hbar\right)
\end{align}
This means that the phase of the wave function oscillates in time with a frequency of the corresponding energy.



In [ ]:
from numpy import *
from scipy import linalg
import matplotlib.pyplot as plt
t = arange(0,20,0.1)
energy = 2
plt.plot(t,real(exp(-1j * energy * t)),label='real part')
plt.plot(t,imag(exp(-1j * energy * t)),label='imaginary part')
plt.plot(t,abs(exp(-1j * energy * t))**2,label='norm')
plt.legend()
plt.show()

__The time dependence of the complete wave function is a modulation of its phase - the real and imaginary components oscillate in time.__

Construction the probability density leads to the expression
\begin{align}
P(x,t) = |\Psi(x) \exp\left(-iEt/\hbar\right)|^2 dx =\exp\left(iEt/\hbar\right) \exp\left(-iEt/\hbar\right) |\Psi(x)|^2 dx  = |\Psi(x)|^2 dx
\end{align}
q.e.d. - we obtain a stationary probability density.

This however only holds for the energy eigenstates. As you know, any linear combinations of eigenstates is itself a solution to the TISE, as per the superposition principle (a combination of solutions to a linear equation is also a solution of it).

If we now superimpose two energy eigenstates 1 and 2 with coefficients $A$ and $B$ that ensure normalization to form a new wave function $\Psi_i(x,t)$
\begin{align}
\Psi_i(x,t) = A \Psi_1(x) \exp\left(-iE_1t/\hbar\right) + B \Psi_2(x) \exp\left(-iE_2t/\hbar\right)
\end{align}
we obtain a superposition state. Let's look at an example:

In [ ]:
a = sqrt(0.75)
b = sqrt(0.25)
w = 1
steps = 100
xgrid = linspace(-w/2,w/2,steps)
prefac = sqrt(2/w)*1/sqrt(steps)
angle = pi * linspace(0,w,steps)/w
psi1 = sin(1*angle)*prefac
psi2 = sin(2*angle)*prefac
print(linalg.norm(psi1),linalg.norm(psi2))
psi = a*psi1 + b*psi2
print(linalg.norm(psi))
plt.plot(xgrid,psi)
plt.show()
plt.plot(xgrid,psi**2)
plt.show()

In [ ]:
#get the expectation values
class expect:
    '''This is the expectation value class.'''
    def __init__(self, wave, xgrid, params):
        self.wave = wave
        self.xgrid = xgrid
        self.ngrid = len(self.wave)
        self.params = params
    
    def x(self):
        xval = sum(conj(self.wave)*self.xgrid*self.wave)
        return xval

    def p(self):
        pgrid = zeros((self.ngrid),dtype=complex)
        h=self.xgrid[1]-self.xgrid[0]
        pgrid= 1j * gradient(self.wave,h)
        #plt.plot(self.xgrid,real(pgrid))
        #plt.plot(self.xgrid,imag(pgrid))
        #plt.plot(self.xgrid,self.wave)
        pval = sum(conj(self.wave)*pgrid)
        return pval

In [ ]:
expectval = expect(psi,xgrid,[w])
print(expectval.x())
print(expectval.p())

Now, this is only a picture of the system at time $t=0$. We know that the complete, time-dependent wave function has a phase that is time-dependent, and for the superposition state, this leads to a time-dependence of observables as the two constituting wave functions overlap differently due to their different phases at different times:


In [ ]:
tfinal = 20
tsteps = 50
au2fs = 0.0241888
tgrid = linspace(0,tfinal,tsteps)
tgridau = linspace(0,tfinal/au2fs,tsteps)
e1 = 1**2 * pi**2 / (2*w**2)
e2 = 2**2 * pi**2 / (2*w**2)
psi1t=zeros((tsteps,steps),dtype=complex)
psi2t=zeros((tsteps,steps),dtype=complex)
psit=zeros((tsteps,steps),dtype=complex)

for i in range(tsteps):
    psi1t[i]=psi1*exp(-1j*e1*tgridau[i])
    psi2t[i]=psi2*exp(-1j*e2*tgridau[i])
    psit[i] = a*psi1*exp(-1j*e1*tgridau[i]) + b*psi2*exp(-1j*e2*tgridau[i])

In [ ]:
factor=5
noplots=tsteps//factor
fig, ax = plt.subplots(noplots,2,figsize=(10,3*noplots),sharex=True)
for i in range(noplots):
    ax[i,0].plot(xgrid,real(psi1t[i*factor]),label='real part')
    ax[i,0].plot(xgrid,imag(psi1t[i*factor]),label='imaginary part')
    #ax[i,0].plot(xgrid,abs(psi1t[i*factor])**2,label='norm')
    #ax[i].legend()
    ax[i,0].text(-0.4,0.17,'t = {:2.1f}fs'.format(tgrid[i*factor]),fontsize=14)
    ax[i,0].set_ylim(bottom=-0.2,top=0.2)
    ax[i,1].plot(xgrid,real(psi2t[i*factor]),label='real part')
    ax[i,1].plot(xgrid,imag(psi2t[i*factor]),label='imaginary part')
    #ax[i,1].plot(xgrid,abs(psi2t[i*factor])**2,label='norm')
    
ax[0,0].set_title(r'$\Psi_1$',fontsize=18)
ax[0,1].set_title(r'$\Psi_2$',fontsize=18)

plt.subplots_adjust(wspace = 0.0)
plt.subplots_adjust(hspace = 0.0)

plt.show()

In [ ]:
factor=5
noplots=tsteps//factor
fig, ax = plt.subplots(noplots,3,figsize=(15,3*noplots),sharex=True)
for i in range(noplots):
    ax[i,0].plot(xgrid,real(psi1t[i*factor]),label='real part')
    ax[i,0].plot(xgrid,imag(psi1t[i*factor]),label='imaginary part')
    #ax[i,0].plot(xgrid,abs(psi1t[i*factor])**2,label='norm')
    #ax[i].legend()
    ax[i,0].text(-0.4,0.17,'t = {:2.1f}fs'.format(tgrid[i*factor]),fontsize=14)
    ax[i,0].set_ylim(bottom=-0.2,top=0.2)
    ax[i,1].plot(xgrid,real(psi2t[i*factor]),label='real part')
    ax[i,1].plot(xgrid,imag(psi2t[i*factor]),label='imaginary part')
    #ax[i,1].plot(xgrid,abs(psi2t[i*factor])**2,label='norm')
    ax[i,2].plot(xgrid,real(psit[i*factor]),label='real part')
    ax[i,2].plot(xgrid,imag(psit[i*factor]),label='imaginary part')
    #ax[i,2].plot(xgrid,abs(psit[i*factor])**2,label='norm')
plt.subplots_adjust(wspace = 0.0)
plt.subplots_adjust(hspace = 0.0)
ax[0,0].set_title(r'$\Psi_1$',fontsize=18)
ax[0,1].set_title(r'$\Psi_2$',fontsize=18)
ax[0,2].set_title(r'$\Psi_i$',fontsize=18)

plt.show()

In [ ]:
factor=5
noplots=tsteps//factor
fig, ax = plt.subplots(noplots,3,figsize=(15,3*noplots),sharex=True,sharey=True)
for i in range(noplots):
    ax[i,0].plot(xgrid,abs(psi1t[i*factor])**2,label='norm')
    ax[i,0].text(-0.4,0.01,'t = {:2.1f}fs'.format(tgrid[i*factor]),fontsize=14)
    ax[i,0].set_ylim(bottom=-0.0,top=0.03)
    ax[i,1].plot(xgrid,abs(psi2t[i*factor])**2,label='norm')
    ax[i,2].plot(xgrid,abs(psit[i*factor])**2,label='norm')
plt.subplots_adjust(wspace = 0.0)
plt.subplots_adjust(hspace = 0.0)
ax[0,0].set_title(r'$|\Psi_1|^2$',fontsize=18)
ax[0,1].set_title(r'$|\Psi_2|^2$',fontsize=18)
ax[0,2].set_title(r'$|\Psi_i|^2$',fontsize=18)

plt.show()

# Task 1

Calculate expectation values of the above superposition state. Plot the expectation values over time and save as a pdf that can be uploaded to moodle. Explain the results.

# Task 2

Generate different superposition states based on
a) different admixtures of two energy eigenstates;
b) increasing number of energy eigenstates in the superposition.
Careful: Make sure the norm is conserved (the coefficients squared sum up to one).

Compare the probability densities and the position expectation values.
What do you notice in case a) and case b)? Provide your answer and one (several) representative plot(s) on moodle.


# Optional task

Start writing a program that can be used to solve the time-independent (TISE) and time-dependent Schödinger equation (TDSE) for the particle in the box. Instead of propagating the analytical solution of the TISE in the TDSE, use the numerical solution of the TISE as in the previous week (wave functions and energies). Include the calculation of energies, and position and momentum expectation values and variance.

You may also include the 2D-box if you wish.

Keep the program general (include a potential term and set it to zero for the particle in the box) so you can extend this to the HO (AHO) and do not write a "one-problem" program (program that only works for one specific task).

Upload your notebook to moodle.